Methods and Experiments
=============

<a id='sec1'></a>

## 1. Experimental Setup

Enumerate all trajectories for each user given the trajectory length (e.g. 3, 4, 5) and the (start, end) POIs.

For each trajectory, compute a score based on the features below:
 * User Interest (time)
 * User interest (frequency)
 * POI Popularity
 * Travelling Cost
 * Trajectory probability based on the transition probabilities between different POI categories and the following rules for choosing a specific POI within certain category:
  * The Nearest Neighbor of the current POI
  * The most Popular POI
  * A random POI choosing with probability proportional to the reciprocal of its distance to current POI
  * A random POI choosing with probability proportional to its popularity

To avoid numerical underflow, use the log of of probabilities instead of the probabilities themselves, 
to avoid zero probabilities, add a smooth value $\epsilon = 10^{-9}$ for each probability.

Plot the scores of generated and actual trajectories for each (user, trajectoryLength, startPOI, endPOI) tuple with some degree of transparency (alpha).

Recommend trajectory with the highest score and measure the performance of recommendation using recall, precision and F1-score.

Optimise parameters in the score function by learning, in this specific case, the cost function could be based on recall, precision or F1-score, we can also control the estimation of transition matrix.

<a id='sec3'></a>

## 3. Scond Experiment Results

Histogram of trajectory length in training set.

<img src='seqlen_hist.png' style='width:600px;height=300px;position:relative;left:-100px'></img>

### 3.1 Changes in Setting

1. the values of parameters/weight vector are drawn from range `[-1, 1]`, and the sum now doesn't necessarily to be 1.

### 3.2 Experiment with single features

<img src='images/feature1.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *first* feature only.

The best F1-score `0.641` was achieved with weigth `0`,
and the worst F1-score `0.582` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *first* feature is the total expected time the user spent at all POIs in a trajectory.
It seems the existance of this feature negative affected the algorithm, 
which is strange as the IJCAI paper argues that capturing the expected time a user spent at POI will improve the accuracy of trajectory recommendation.

<img src='images/feature2.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *second* feature only.

The best F1-score `0.641` was achieved with weigth `0`,
and the worst F1-score `0.577` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *second* feature is the total number of visit (of a user) of all POIs in a trajectory.
Similar to the *first* feature, it seems the existance of this feature negative affected the algorithm, 
which is also strange as experiments from the IJCAI paper show that capturing a user's visiting frequency of POI will improve the accuracy of trajectory recommendation, though less that capturing visit time duration, but still much better than greedy and random selection strategies.

<img src='images/feature3.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *third* feature only.

The best F1-score `0.641` was achieved with weigth `[0, 0.05, ..., 0.95, 1]`,
and the worst F1-score `0.577` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *third* feature is the total POI popularity (i.e. # of visit of a POI by all users) of all POIs in a trajectory.
It's clear from the experiment that a positive weight of this feature help the recommendation algorithm.

<img src='images/feature4.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *fourth* feature only.

The best F1-score `0.718` was achieved with weigth `[0.05, 0.1, ..., 0.95, 1]`,
and the worst F1-score `0.596` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *fourth* feature is the negative (i.e. multiple -1) of total travelling cost (i.e. travel time, including visit time duration at POI and time spent from one POI to another in the trajectory) for a user of a trajectory.
It's clear the algorithm dislikes trajectories which cost a lot of time.

<img src='images/feature5.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *fifth* feature only.

The best F1-score `0.682` was achieved with weigth `[0.05, 0.1, ..., 0.95, 1]`,
and the worst F1-score `0.581` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *fifth* feature is the probability of a recommended trajectory based on the transition probabilities between POI categories and the nearest neighbor rule for choosing a specific POI within a certain category.
It's clear the algorithm dislikes far neighbors.

<img src='images/feature6.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *sixth* feature only.

The best F1-score `0.66` was achieved with weigth `[0.05, 0.1, ..., 0.95, 1]`,
and the worst F1-score `0.586` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *sixth* feature is the probability of a recommended trajectory based on the transition probabilities between POI categories and the most popular POI rule for choosing a specific POI within a certain category. It's clear the algorithm dislikes non-popular POIs.

<img src='images/feature7.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *seventh* feature only.

The best F1-score `0.641` was achieved with weigth `0`,
and the worst F1-score `0.589` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *seventh* feature is the probability of a recommended trajectory based on the transition probabilities between POI categories and a rule below for choosing a specific POI within a certain category.  
*Rule: choose a random POI with probability proportional to the reciprocal of its distance to current POI.*  
Similar to the *fifth* feature which utilises the nearest neighbor rule, the algorithm doesn't like far neighbors.

<img src='images/feature8.png' style='width:600px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with the *last* feature only.

The best F1-score `0.675` was achieved with weigth `[0.05, 0.1, ..., 0.95, 1]`,
and the worst F1-score `0.583` was achieved with weight `[-1.0, -0.95, ..., -0.1, -0.05]`.

The *last* feature is the probability of a recommended trajectory based on the transition probabilities between POI categories and a rule below for choosing a specific POI within a certain category.  
*Rule: choose a random POI with probability proportional to its popularity.*  
Similar to the *sixth* feature which utilises the most popular POI rule, the algorithm doesn't like non-popular POIs. 

### 3.3 Experiment with a large number of random weights

<img src='images/random_scatter1e4.png' style='width:600px;height:400px;position:relative;left:-100px'></img>

The above scatter plot shows the searching process with `10,000` random weight vectors.

Each weight vector was generated by command `weights = numpy.random.uniform(-1, 1, 8)`.  
The *best* F1-score is `0.737`, 
achieved by weight vector  
`[-0.409032, 0.840751, 0.539495, 0.210137, 0.959562, 0.391944, 0.53334, -0.931497]`  
The *worst* F1-score is `0.577`,
achieved by weight vector  
`[0.451991, -0.038394, -0.132963, -0.735366, 0.404607, 0.894026, -0.918393, 0.759964]`

<img src='images/random_hist1e4.png' style='width:600px;height:400px;position:relative;left:-100px'></img>

The above histogram shows all the F1-scores using these random weights, there are clear pattern in the F1-scores.  
But it's hard to visualise 8-dimentional data, the following scatter plots shows the relationship between F1-score and the weight of each feature.

<img src='images/random_scattersub.png' style='width:900px;height:1200px;position:relative;left:-100px'></img>

<a id='sec2'></a>

## 2. First Experiment Results

### 2.1 Basic computing steps

1. Split actual trajectories input two parts, one for training, the other for testing.  
   Concretely, For each user, consider all the trajectories with length `3`, `4` and `5`, pick one for testing set and put all others into training set.
   
1. Use trajectories in training set to compute (MLE) a transition matrix where element `[i, j]` denotes the transition probability from POI category `i` to POI category `j`.

1. For each trajectory $T$ in training set, enumerate all possible trajectories that satisfy the following requirements:
 * The trajectory length is the same as that of $T$
 * The start/end POI are the same as those of $T$
 * No sub-tour exists
 
1. Compute the `8` scores [described above](#sec1), rescale each score into range `[-1, 1]`, compute the weighted sum of these score to get a single score (weights are normalised so that they are in range `[0, 1]` and their sum is `1`)
 
1. Choose the trajectory with the highest score $T^*$ and compute F1 score as follows:  
 * recall = $\frac{|T^* \cap T|}{|T|}$  
 * precision = $\frac{|T^* \cap T|}{T^*}$  
 * F1-score = $\frac{2 \times \text{recall} \times \text{precision}}{\text{recall} + \text{precision}}$
 
1. Compute the mean F1 score for all trajectory $T$ in the training set.

1. Use coordinate-wise grid search to find an good weight vector such that the mean F1 score is as large as possible.

### 2.2 Some experimental results

Random weights, use 10 random weight vector, the mean F1 scores are as follows:  
`[0.717, 0.705, 0.665, 0.719, 0.698, 0.719, 0.704, 0.724, 0.693, 0.665]`,  mean = `0.701`, std = `0.02`

Some arbitrary weights:  
`[[1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8], # avgF1: 0.712`  
 `[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], # avgF1: 0.712`  
 `[0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], # avgF1: 0.707`  
 `[0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], # avgF1: 0.700`  
 `[0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1]] # avgF1: 0.708`

Coordinate-wise grid search, start from `[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]`, for each coordinate,  
search a value in [0, 0.05, 0.1, 0.15, 0.2, ..., 0.95, 1] to maximize the mean F1 score while fixing other coordinates.

Fix the above weights except the *first* one, a weight vector with best F1-score is `[0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord1.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *second* one, a weight vector with best F1-score is `[0.15, 0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord2.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *third* one, a weight vector with best F1-score is `[0.15, 0, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord3.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *fourth* one, a weight vector with best F1-score is `[0.15, 0, 0.05, 0.25, 0.1, 0.1, 0.1, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord4.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *fifth* one, a weight vector with best F1-score is `[0.15, 0, 0.05, 0.25, 0.4, 0.1, 0.1, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord5.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *sixth* one, a weight vector with best F1-score is `[0.15, 0, 0.05, 0.25, 0.4, 0, 0.1, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord6.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *seventh* one, a weight vector with best F1-score is `[0.15, 0, 0.05, 0.25, 0.4, 0, 0, 0.1]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord7.png' style='width:600px;position:relative;left:-100px'></img>

Fix the above weights except the *last* one, a weight vector with best F1-score is `[0.15, 0, 0.05, 0.25, 0.4, 0, 0, 0.9]`, scatter plot of F1-scores in the searching process is shown below.

<img src='images/coord8.png' style='width:600px;position:relative;left:-100px'></img>

Finally, the weight vector with best F1-score `[0.15, 0, 0.05, 0.25, 0.4, 0, 0, 0.9]` is applied to the testing set, and the mean F1-score in testing set is `0.727` (vs. `0.671` in ijcai15 paper)